In [ ]:
# !wget 'https://www.robots.ox.ac.uk/~maxbain/frozen-in-time/data/MSRVTT.zip' -P data

In [ ]:
# !unzip '/content/drive/MyDrive/Final Project/data/MSRVTT.zip' -d data

In [ ]:
! find '/content/drive/MyDrive/Final Project/data/MSRVTT/videos/gifs' -name '*.gif' | wc -l

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json 

with open('/content/drive/MyDrive/Final Project/data/MSRVTT/annotation/MSR_VTT.json') as f:
    data = json.load(f)


In [ ]:
ann_note = []
ann_name = []
for i in range(len(data['annotations'])):
  if data['annotations'][i]['image_id'] in ann_name:
    continue
  else:
    ann_note.append((data['annotations'][i]['image_id'],data['annotations'][i]['caption']))
    ann_name.append(data['annotations'][i]['image_id'])
print(len(ann_name),len(ann_note))

10000 10000


In [ ]:
from natsort import natsorted
ann_data = natsorted(ann_note, key=lambda x:x[0])
ann_video = [ann_data[i][1] for i in range(len(ann_data))]
print(ann_video)

In [ ]:
import os
video_idx = os.listdir('/content/drive/MyDrive/Final Project/data/MSRVTT/videos/all')
gifs_idx = os.listdir('/content/drive/MyDrive/Final Project/data/MSRVTT/videos/gifs')

In [ ]:
! pip install imageio-ffmpeg
! sudo pip3 install imageio==2.4.1
! pip install imageio
! pip3 install imageio==2.4.1
! pip install --upgrade imageio-ffmpeg
! sudo pip3 install imageio==2.4.1
! pip install moviepy --ignore-installed

In [ ]:
tensor_idx = os.listdir('/content/drive/MyDrive/Final Project/data/MSRVTT/videos/tensor')
trans_idx = []
notrans_idx = []
for i in range(10000):
  if f'video{i}.pt' in tensor_idx:
    trans_idx.append(f'video{i}.pt')
  else:
    notrans_idx.append(f'video{i}.pt')
    torch.save(video_data[i][0],f'/content/drive/MyDrive/Final Project/data/MSRVTT/videos/tensor/video{i}.pt')

In [ ]:
print(notrans_idx)

In [ ]:
from moviepy.editor import VideoFileClip

In [ ]:
%cd '/content/drive/MyDrive/Final Project/data/MSRVTT/videos'
#os.makedirs('./gifs', exist_ok=True)
for i in notrans_idx[0:32]:
  videoClip = VideoFileClip(os.path.join('./all',i))
  videoClip.write_gif(os.path.join('./gifs',os.path.splitext(i)[0]+'.gif'))

In [ ]:
from pathlib import Path
from torch.optim import Adam
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from functools import partial
from torch.utils import data
import os
import torch

CHANNELS_TO_MODE = {
    1 : 'L',
    3 : 'RGB',
    4 : 'RGBA'
}

def seek_all_images(img, channels = 3):
    assert channels in CHANNELS_TO_MODE, f'channels {channels} invalid'
    mode = CHANNELS_TO_MODE[channels]

    i = 0
    while True:
        try:
            img.seek(i)
            yield img.convert(mode)
        except EOFError:
            break
        i += 1


def gif_to_tensor(path, channels = 3, transform = T.ToTensor()):
    img = Image.open(path)
    tensors = tuple(map(transform, seek_all_images(img, channels = channels)))
    return torch.stack(tensors, dim = 1)

def identity(t, *args, **kwargs):
    return t

def cast_num_frames(t, *, frames):
    f = t.shape[1]

    if f == frames:
        return t

    if f > frames:
        return t[:, :frames]

    return F.pad(t, (0, 0, 0, 0, 0, frames - f))

class Dataset(data.Dataset):
    def __init__(
        self,
        folder,
        image_size = 32,
        channels = 3,
        num_frames = 16,
        horizontal_flip = False,
        force_num_frames = True,
        exts = ['gif']
    ):
        super().__init__()
        self.folder = folder
        self.image_size = image_size
        self.channels = channels
        idx = natsorted(os.listdir(folder))
        self.paths = [Path(os.path.join(folder,idx[i])) for i in range(len(idx))]

        self.cast_num_frames_fn = partial(cast_num_frames, frames = num_frames) if force_num_frames else identity

        self.transform = T.Compose([
            T.Resize(image_size),
            T.RandomHorizontalFlip() if horizontal_flip else T.Lambda(identity),
            T.CenterCrop(image_size),
            T.ToTensor()
        ])

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        path = self.paths[index]
        tensor = gif_to_tensor(path, self.channels, transform = self.transform)
        return [self.cast_num_frames_fn(tensor),ann_video[index]]

In [ ]:
video_data = Dataset('/content/drive/MyDrive/Final Project/data/MSRVTT/videos/gifs')

In [ ]:
! find '/content/drive/MyDrive/Final Project/data/MSRVTT/videos/tensor' -name '*.pt' | wc -l

10000


In [ ]:
tensor_idx = os.listdir('/content/drive/MyDrive/Final Project/data/MSRVTT/videos/tensor')
pro_idx = [f'video{i}.pt' for i in range(10000)]
rep = []
for j in tensor_idx:
  if j in pro_idx:
    continue
  else:
    os.remove(os.path.join('/content/drive/MyDrive/Final Project/data/MSRVTT/videos/tensor',j))

In [ ]:
import os
import torch
from torch.utils import data
class TensorDataset(data.Dataset):
    def __init__(
        self,
        folder
    ):
        super().__init__()
        self.folder = folder
        idx = pro_idx = [f'video{i}.pt' for i in range(10000)]
        self.paths = [os.path.join(folder,idx[i]) for i in range(len(idx))]

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        path = self.paths[index]
        tensor = torch.load(path)[:,:5,:,:]
        return [tensor,ann_video[index]]

tensor_data = TensorDataset('/content/drive/MyDrive/Final Project/data/MSRVTT/videos/tensor')

In [ ]:
len(tensor_data)

10000

In [ ]:
tensor_data[833][1]

'a girl is explaining about travel bag'

In [ ]:
!pip install video-diffusion-pytorch
!pip install -U sentence-transformers
!pip install pytorch-transformers

In [ ]:
import torch
from video_diffusion_pytorch import Unet3D, GaussianDiffusion, Trainer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Unet3D(
    dim = 64,
    use_bert_text_cond = True,  # this must be set to True to auto-use the bert model dimensions
    dim_mults = (1, 2, 4, 8),
).to(device)


diffusion = GaussianDiffusion(
    model,
    image_size = 32,    # height and width of frames
    num_frames = 5,     # number of video frames
    timesteps = 1000,  # number of steps
    text_use_bert_cls = True,  
    loss_type = 'l1'    # L1 or L2
).to(device)

In [ ]:
import copy
from torch.cuda.amp import autocast, GradScaler
from einops import rearrange


class EMA():
    def __init__(self, beta):
        super().__init__()
        self.beta = beta

    def update_model_average(self, ma_model, current_model):
        for current_params, ma_params in zip(current_model.parameters(), ma_model.parameters()):
            old_weight, up_weight = ma_params.data, current_params.data
            ma_params.data = self.update_average(old_weight, up_weight)

    def update_average(self, old, new):
        if old is None:
            return new
        return old * self.beta + (1 - self.beta) * new


def num_to_groups(num, divisor):
    groups = num // divisor
    remainder = num % divisor
    arr = [divisor] * groups
    if remainder > 0:
        arr.append(remainder)
    return arr

In [ ]:
from torchvision import transforms as T, utils
def video_tensor_to_gif(tensor, path, duration = 120, loop = 0, optimize = True):
    images = map(T.ToPILImage(), tensor.unbind(dim = 1))
    first_img, *rest_imgs = images
    first_img.save(path, save_all = True, append_images = rest_imgs, duration = duration, loop = loop, optimize = optimize)
    return images

In [ ]:
from torch.optim import Adam
batch = 32
ema = EMA(0.995)
ema_model = copy.deepcopy(diffusion)
train_data = torch.utils.data.DataLoader(tensor_data, batch_size=batch, shuffle=True, num_workers = 2)
optimizer = Adam(diffusion.parameters(), lr = 1e-4)
scaler = GradScaler(enabled = True)

In [ ]:
data = torch.load('/content/drive/MyDrive/Final Project/data/MSRVTT/videos/results/model-current.pt')

step = data['step']
diffusion.load_state_dict(data['model'])
ema_model.load_state_dict(data['ema'])
scaler.load_state_dict(data['scaler'])

In [ ]:
step

185

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F
epochs = 500
for epoch in range(step+1,epochs):
  
  for train_idx in tqdm(train_data):
    videos, annotation = train_idx
    with autocast(enabled = True):
      loss = diffusion(
          videos.to(device),
          cond = annotation
      )
      scaler.scale(loss).backward()
  print(f"Epoch: {epoch}, Loss: {loss.item():.3f}")
  scaler.step(optimizer)
  scaler.update()
  optimizer.zero_grad()

  data = {
      'step': epoch,
      'model': diffusion.state_dict(),
      'ema': ema_model.state_dict(),
      'scaler': scaler.state_dict()
  }
  torch.save(data, '/content/drive/MyDrive/Final Project/data/MSRVTT/videos/results/model-current.pt')

  if epoch % 10 == 0:
    if epoch < 2000:
      ema_model.load_state_dict(diffusion.state_dict())
    ema.update_model_average(ema_model, diffusion)

  if epoch != 0 and epoch % 5 == 0:
    milestone = epoch // 5
    num_samples = 16
    batches = num_to_groups(num_samples, batch)

    all_videos_list = list(map(lambda n: ema_model.sample(batch_size=n, cond=annotation), batches))
    all_videos_list = torch.cat(all_videos_list, dim = 0)

    all_videos_list = F.pad(all_videos_list, (2, 2, 2, 2))

    one_gif = rearrange(all_videos_list, '(i j) c f h w -> c f (i h) (j w)', i = 4)
    video_path = f'/content/drive/MyDrive/Final Project/data/MSRVTT/videos/results/output{milestone}.gif'
    video_tensor_to_gif(one_gif, video_path)

    print(f'Succefully save output{milestone}')

  0%|          | 0/313 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

...

100%|██████████| 313/313 [28:17<00:00,  5.42s/it]


Epoch: 200, Loss: 0.177


sampling loop time step: 100%|██████████| 1000/1000 [03:36<00:00,  4.62it/s]


Succefully save output40


 46%|████▋     | 145/313 [10:00<13:46,  4.92s/it]

In [ ]:
import torch
from video_diffusion_pytorch import Unet3D, GaussianDiffusion


videos = torch.randn(3, 3, 5, 32, 32,device=device) # video (batch, channels, frames, height, width)

text = [
    'a whale breaching from afar',
    'young girl blowing out candles on her birthday cake',
    'fireworks with blue and green sparkles'
]

loss = diffusion(videos, cond = text)
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
sampled_videos = diffusion.sample(cond = text, cond_scale = 2)
sampled_videos.shape # (3, 3, 5, 32, 32)

sampling loop time step: 100%|██████████| 1000/1000 [01:49<00:00,  9.17it/s]


torch.Size([3, 3, 5, 32, 32])

In [ ]:
video_tensor_to_gif(sampled_videos[2],'/content/drive/MyDrive/Final Project/gif.gif')